# 2.4 Maxwell's Equations

[Peter Monk: "Finite Elements for Maxwell's Equations"]

## Magnetostatic field generated by a permanent magnet

magnetic flux $B$, magnetic field $H$, given magnetization $M$:

$$
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Div}{div}
B = \mu (H + M), \quad \Div B = 0, \quad \Curl H = 0
$$

Introducing a vector-potential $A$ such that $B = \Curl A$, and putting equations together we get

$$
\Curl \mu^{-1} \Curl A = \Curl M
$$

In weak form: Find $A \in H(\Curl)$ such that

$$
\int \mu^{-1} \Curl A \Curl v = \int M \Curl v \qquad  \forall \, v \in H(\Curl)
$$

Usually, the permeability $\mu$ is given as 
$\mu = \mu_r \mu_0$, with $\mu_0 = 4 \pi 10^{-7}$ the permeability of vacuum.

In [1]:
from ngsolve import *
from netgen.csg import *
import netgen.gui

Geometric model and meshing of a bar magnet:

In [12]:
box = OrthoBrick(Pnt(-3,-3,-3),Pnt(3,3,3)).bc("outer")
magnet = Cylinder(Pnt(-1,0,0),Pnt(1,0,0), 0.3) * OrthoBrick(Pnt(-1,-3,-3),Pnt(1,3,3))
air = box - magnet

geo = CSGeometry()
geo.Add (air.mat("air"), transparent=True) 
geo.Add (magnet.mat("magnet").maxh(1), col=(0.3,0.3,0.1))
geo.Draw()
mesh = Mesh(geo.GenerateMesh(maxh=2, curvaturesafety=1))
mesh.Curve(3)

In [13]:
mesh.GetMaterials(), mesh.GetBoundaries()

(('air', 'magnet'),
 ('outer',
  'outer',
  'outer',
  'outer',
  'outer',
  'outer',
  'default',
  'default',
  'default'))

Define space, forms and preconditioner.

* To obtain a regular system matrix, we regularize by adding a very small $L_2$ term.
* We solve magnetostatics, so we can gauge by adding and arbitrary gradient field. A cheap possibility is to delete all basis-functions which are gradients (flag 'nograds')

In [14]:
fes = HCurl(mesh, order=3, dirichlet="outer", nograds=True)
print ("ndof =", fes.ndof)
u,v = fes.TnT()

from math import pi
mu0 = 4*pi*1e-7
mur = CoefficientFunction( [1000 if mat== "magnet" else 1 
                            for mat in mesh.GetMaterials()]) 

a = BilinearForm(fes)
a += 1/(mu0*mur)*curl(u)*curl(v)*dx + 1e-8/(mu0*mur)*u*v*dx
c = Preconditioner(a, "bddc")

f = LinearForm(fes)
mag = CoefficientFunction((1,0,0)) * \
    CoefficientFunction( [1 if mat == "magnet" else 0 for mat in mesh.GetMaterials()])
f += SymbolicLFI(mag*curl(v), definedon=mesh.Materials("magnet"))

ndof = 21439


Assemble system and setup preconditioner using task-parallelization:

In [16]:
with TaskManager():
    a.Assemble()
    f.Assemble()

Finally, declare GridFunction and solve by preconditioned CG iteration:

In [18]:
gfu = GridFunction(fes)
with TaskManager():
    solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=c.mat)

it =  0  err =  0.004840564319042928
it =  1  err =  0.0028777467354534018
it =  2  err =  0.0018540826001404265
it =  3  err =  0.001201924591628041
it =  4  err =  0.000806914749021638
it =  5  err =  0.0005450296846235357
it =  6  err =  0.0003675266044562537
it =  7  err =  0.00024345894519279366
it =  8  err =  0.0001435746661404215
it =  9  err =  9.620650815911703e-05
it =  10  err =  5.873314058016385e-05
it =  11  err =  3.586599439923319e-05
it =  12  err =  2.236083879867451e-05
it =  13  err =  1.3049751993777922e-05
it =  14  err =  8.434902961346295e-06
it =  15  err =  5.3998054883886365e-06
it =  16  err =  3.0315968419612386e-06
it =  17  err =  1.826457133757768e-06
it =  18  err =  1.1434143259772091e-06
it =  19  err =  7.016189041097084e-07
it =  20  err =  4.5426431119939464e-07
it =  21  err =  2.6888162144515283e-07
it =  22  err =  1.585512744829188e-07
it =  23  err =  9.730336619012959e-08
it =  24  err =  5.622498588543289e-08
it =  25  err =  3.375384192196

In [19]:
# the vector potential is not supposed to look nice
Draw (gfu, mesh, "vector-potential", draw_surf=False)

Draw (curl(gfu), mesh, "B-field", draw_surf=False)
Draw (1/(mu0*mur)*curl(gfu)-mag, mesh, "H-field", draw_surf=False)